In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
import random
import pandas
from sklearn.metrics.pairwise import pairwise_distances

In [131]:
def generate_dataset(filename):
    col_names = ['ts','user','x_coordinate','y_coordinate','color', 'label']
    data = pandas.read_csv(filename, names = col_names, header=None, skiprows=1)
#     print(type(data))
    
#     ts = data.ts.values
    
#     print(type(ts))
    return data

In [178]:
num_labels = 2
def extract_data(filename, num_labels):
    data = generate_dataset(filename)
    dim = data.ts.values.shape[0]
    print(dim)
    #type(data.x_coordinate.values[1])
    dataset_x = np.concatenate((data.ts.values.reshape(dim,1),
                              data.user.values.reshape(dim,1),
                              data.x_coordinate.values.reshape(dim,1),
                              data.y_coordinate.values.reshape(dim,1),
                              data.color.values.reshape(dim,1)), axis=1)
    print(dataset_x.shape)
    dataset_y = np.zeros((dim,num_labels))
    for i in range(dim):
        dataset_y[i][data.label.values[i]] = 1
    print(dataset_y.shape)
    return dataset_x, dataset_y

In [191]:
train_x, train_y = extract_data('four_blocks.txt', num_labels)
# valid_x, valid_y = extract_data('validation.txt', num_labels)
test_x, test_y = extract_data('test_blocks.txt', num_labels)
print(train_x)

60000
(60000, 5)
(60000, 2)
20000
(20000, 5)
(20000, 2)
[[ 0  0  0  0  0]
 [ 0  0  0  1  0]
 [ 0  0  0  2  0]
 ...
 [ 0  0 99 97  0]
 [ 0  0 99 98  0]
 [ 0  0 99 99  0]]


In [188]:
def build_model():
    x = tf.placeholder(tf.float32, shape=(None, 5), name='x')    
    y = tf.placeholder(tf.float32, shape=(None, num_labels), name='y') 
    
    W1 = tf.Variable(tf.random_normal([5, 128], stddev = 0.03), name='W1')
    b1 = tf.Variable(tf.random_normal([128]), name='b1')
    
#     W2 = tf.Variable(tf.random_normal([128, 128], stddev = 0.5), name='W2')
#     b2 = tf.Variable(tf.random_normal([128]), name='b2')
    
    W3 = tf.Variable(tf.random_normal([128, num_labels], stddev=0.03), name='W3')
    b3 = tf.Variable(tf.random_normal([num_labels]), name='b3')

    
    hidden_out = tf.add(tf.matmul(x, W1), b1)
    hidden_out = tf.nn.relu(hidden_out)
    
#     hidden_out2 = tf.add( tf.matmul(hidden_out, W2), b2 )
#     hidden_out2 = tf.nn.relu(hidden_out2)
    
    logits = tf.add(tf.matmul(hidden_out, W3), b3)
    prediction = tf.nn.softmax(logits)
    y_clipped = tf.clip_by_value(prediction, 1e-10, 0.9999999)
    
    loss = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1 - y) * tf.log(1 - y_clipped), axis=1))
#     loss = tf.losses.cosine_distance(y, prediction, axis = 1)
    
    train_op = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(loss)
    init = tf.global_variables_initializer()

    correct_prediction = tf.equal(tf.argmax(prediction, 1, name="Argmax_Pred"), tf.argmax(y, 1, name="Y_Pred"), name="Correct_Pred")
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32, name="Cast_Corr_Pred"), name="Accuracy")
    
    return x, y, init, train_op, loss, accuracy

In [189]:
def run(x, y, init, train_op, loss, accuracy):
    #define session
#     train_distances = pairwise_distances(train_x, metric = 'cosine')
#     test_distances = pairwise_distances(train_x, metric = 'cosine')
    MAX_THRESH = 10
    with tf.Session() as sess:
        sess.run(init)
        feed_dict = {x: train_x, y: train_y}
        valid_dict = {x: valid_x, y: valid_y}
        test_dict = {x: test_x, y: test_y}
        
        error_increases = 0
        old_valid_accuracy = 0
        for i in range(1000):
            #print('EPOCH', i)
            _, loss_val = sess.run([train_op, loss], feed_dict)
#             valid_accuracy = sess.run(accuracy, valid_dict)
# #             print("VALID ACC: ", valid_accuracy)
#             if (valid_accuracy < old_valid_accuracy):
#                 error_increases += 1
#                 if error_increases > MAX_THRESH:
#                     break
#             else:
#                 error_increases = 0
            
#             old_valid_accuracy = valid_accuracy
            
            #print('DONE WITH EPOCH')
            #print("loss:", loss_val)

        print("Training Accuracy:", sess.run( accuracy, feed_dict))
        print("Testing Accuracy:", sess.run( accuracy, test_dict))

In [ ]:
x, y, init, train_op, loss, accuracy = build_model()
run(x, y, init, train_op, loss, accuracy)

In [193]:

x = tf.placeholder(tf.float32, shape=(None, 5), name='x')    
y = tf.placeholder(tf.int64, shape=(None, num_labels), name='y') 

layer1 = tf.contrib.layers.fully_connected(x, 128, tf.nn.relu)
predictions = tf.contrib.layers.fully_connected(layer1, num_labels, tf.nn.softmax)
loss = tf.losses.softmax_cross_entropy(onehot_labels = y, logits = predictions)
correct_prediction = tf.equal(tf.argmax(predictions, 1, name="Argmax_Pred"), tf.argmax(y, 1, name="Y_Pred"), name="Correct_Pred")
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32, name="Cast_Corr_Pred"), name="Accuracy")

train_op = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(loss)


#define session
sess = tf.Session()
sess.run(tf.global_variables_initializer())
feed_dict = {x: train_x, y: train_y}


for i in range(100):
    print('EPOCH', i)
    _, loss_val = sess.run([train_op, loss], feed_dict)
    print('DONE WITH EPOCH')
    print("loss:", loss_val)

print("Accuracy:", sess.run( accuracy, feed_dict = {x: train_x, y: train_y}))
    
    

EPOCH 0
DONE WITH EPOCH
loss: 1.5720711
EPOCH 1
DONE WITH EPOCH
loss: 1.5418348
EPOCH 2
DONE WITH EPOCH
loss: 1.4929717
EPOCH 3
DONE WITH EPOCH
loss: 1.468345
EPOCH 4
DONE WITH EPOCH
loss: 1.4578428
EPOCH 5
DONE WITH EPOCH
loss: 1.4527224
EPOCH 6
DONE WITH EPOCH
loss: 1.4474506
EPOCH 7
DONE WITH EPOCH
loss: 1.4421428
EPOCH 8
DONE WITH EPOCH
loss: 1.4371489
EPOCH 9
DONE WITH EPOCH
loss: 1.4350188
EPOCH 10
DONE WITH EPOCH
loss: 1.4493898
EPOCH 11
DONE WITH EPOCH
loss: 1.5194391
EPOCH 12
DONE WITH EPOCH
loss: 1.4289603
EPOCH 13
DONE WITH EPOCH
loss: 1.446091
EPOCH 14
DONE WITH EPOCH
loss: 1.5298197
EPOCH 15
DONE WITH EPOCH
loss: 1.4362342
EPOCH 16
DONE WITH EPOCH
loss: 1.4927328
EPOCH 17
DONE WITH EPOCH
loss: 1.5227529
EPOCH 18
DONE WITH EPOCH
loss: 1.422243
EPOCH 19
DONE WITH EPOCH
loss: 1.4389552
EPOCH 20
DONE WITH EPOCH
loss: 1.5361227
EPOCH 21
DONE WITH EPOCH
loss: 1.4411899
EPOCH 22
DONE WITH EPOCH
loss: 1.5159793
EPOCH 23
DONE WITH EPOCH
loss: 1.4766953
EPOCH 24
DONE WITH EPOCH
loss

In [10]:
def loss( x, labels, alpha, weight ):
    total = 0
    x_arr = x.eval()
    weights_arr = weight.eval()
    pixel_count = dict()
    
    for i in range(10000):
        label = labels[i]
        if pixel_count.get(label) is None:
            pixel_count[label] = 0
        pixel_count[label] += 1
    
    distances = (pairwise_distances(x_arr, metric = 'cosine') + 1) / 2
    for i in range(distances.shape[0]):
        for j in range(i, distances.shape[1]):
            value = ( weights[i] * weights[j] ) / pixel_count(labels[i]) 
            if (labels[i] == labels[j]):
                total +=  value*(1 - distances[i][j])
            else:
                total += (distances[i][j] - alpha)

NameError: name 'data' is not defined